# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction

In this [Kaggle competition](https://www.kaggle.com/c/stanford-covid-vaccine/overview) we try to develop models and design rules for RNA degradation. As the overview of the competition states:

>mRNA vaccines have taken the lead as the fastest vaccine candidates for COVID-19, but currently, they face key potential limitations. One of the biggest challenges right now is how to design super stable messenger RNA molecules (mRNA). Conventional vaccines (like your seasonal flu shots) are packaged in disposable syringes and shipped under refrigeration around the world, but that is not currently possible for mRNA vaccines.
>
>Researchers have observed that RNA molecules have the tendency to spontaneously degrade. This is a serious limitation--a single cut can render the mRNA vaccine useless. Currently, little is known on the details of where in the backbone of a given RNA is most prone to being affected. Without this knowledge, current mRNA vaccines against COVID-19 must be prepared and shipped under intense refrigeration, and are unlikely to reach more than a tiny fraction of human beings on the planet unless they can be stabilized.

<img src="images/banner.png" width="1000" style="margin-left: auto; margin-right: auto;"> 

The model should predict likely degradation rates at each base of an RNA molecule. The training data set is comprised of over 3000 RNA molecules and their degradation rates at each position.

# Install necessary packages

We can install the necessary package by either running `pip install --user <package_name>` or include everything in a `requirements.txt` file and run `pip install --user -r requirements.txt`. Since we only need one extra package here we can use the former command.

> NOTE: Do not forget to use the `--user` argument. It is necessary if you want to use Kale to transform this notebook into a Kubeflow pipeline

In [ ]:
# !pip install --user pandas

# Imports

In this section we import the packages we need for this example. Make it a habbit to gather your imports in a single place. It will make your life easier if you are going to transform this notebook into a Kubeflow pipeline using Kale.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

# Project hyper-parameters

In this cell, we define the different hyper-parameters. Defining them in one place makes it easier to experiment with their values and also facilitates the execution of HP tuning experiments using Kale and Katib.

In [ ]:
# Hyper-parameters
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
EMBED_DIM = 100
HIDDEN_DIM = 128
DROPOUT = .5
SP_DROPOUT = .3
TRAIN_SEQUENCE_LENGTH = 107

Set random seed for reproducibility.

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

# Load and preprocess data

In this section, we load and process the data set to get in a ready-to-use form by the model. First, let us load and analyze the data.

## Load data

The data are in `json` format, thus, we use the handy `read_json` pandas method. There is one train data set and two test sets (one public and one private).

In [ ]:
train_df = pd.read_json("data/train.json", lines=True)
test_df = pd.read_json("data/test.json", lines=True)

We also load the `sample_submission.csv` file, which will prove handy when we will be creating our submission to the competition.

In [ ]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")

Let us now explore the data, their dimensions and what each column mean. To this end, we use the pandas `head` method to visualize a small sample (five rows by default) of our data set.

In [ ]:
train_df.head()

We see a lot of strange entries, so, let us try to see what they are:

* `sequence`: An 107 characters long string in Train and Public Test (130 in Private Test), which describes the RNA sequence, a combination of A, G, U, and C for each sample.
* `structure`: An 107 characters long string in Train and Public Test (130 in Private Test), which is a compination of `(`, `)`, and `.` characters that describe whether a base is estimated to be paired or unpaired. Paired bases are denoted by opening and closing parentheses (e.g. (....) means that base 0 is paired to base 5, and bases 1-4 are unpaired).
* `predicted_loop_type`: An 107 characters long string, which describes the structural context (also referred to as 'loop type') of each character in sequence. Loop types assigned by bpRNA from Vienna RNAfold 2 structure. From the bpRNA_documentation: `S`: paired "Stem" `M`: Multiloop `I`: Internal loop `B`: Bulge `H`: Hairpin loop `E`: dangling End `X`: eXternal loop.

Then, we have `signal_to_noise`, which is quality control feature. It records the measurements relative to their errors; the higher value the more confident measurements are.

The `*_error_*` columns calculate the errors in experimental values obtained in corresponding `reactivity` and `deg_*` columns.

The last five columns (i.e., `recreativity` and `deg_*`) are out depended variables, our targets. Thus, for every base in the molecule we should predict five different values.

The `bpps` folder stands for Base Pairing Probabilities. In contains numpy arrays that correspond to the Base Pairing Probability Matrix (BPPM). The `bpps` are pre-calculated for each sequence. They are matrices of base pair probabilities. Biophysically speaking, this matrix gives the probability that each pair of nucleotides, in the RNA, forms a base pair (given a particular model of RNA folding). Thus, let us provide some helper functions to load these arrays.

In [ ]:
def read_bpps_sum(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"data/bpps/{mol_id}.npy").max(axis=1))
    return bpps_arr


def read_bpps_max(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"data/bpps/{mol_id}.npy").sum(axis=1))
    return bpps_arr


def read_bpps_nb(df):
    # normalized non-zero number
    # from https://www.kaggle.com/symyksr/openvaccine-deepergcn 
    bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
    bpps_nb_std = 0.08914   # std of bpps_nb across all training data
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps = np.load(f"data/bpps/{mol_id}.npy")
        bpps_nb = (bpps > 0).sum(axis=0) / bpps.shape[0]
        bpps_nb = (bpps_nb - bpps_nb_mean) / bpps_nb_std
        bpps_arr.append(bpps_nb)
    return bpps_arr

These are the main columns we care about. For more details, visit the competition [info](https://www.kaggle.com/c/stanford-covid-vaccine/data).

## Preprocess data

We are now ready to preprocess the data set. First, we define the symbols that encode certain features (e.g. the base symbol or the structure), the features and the target variables.

In [ ]:
symbols = "().ACGUBEHIMSX"
feat_cols = ['sequence', 'structure', 'predicted_loop_type']
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_Mg_50C', 'deg_error_pH10', 'deg_error_50C']

In order to encode values like strings or characters and feed them to the neural network, we need to tokenize them. The `Tokenizer` class will assign a number to each character.

In [ ]:
tokenizer = Tokenizer(char_level=True, filters="")
tokenizer.fit_on_texts(symbols)

Moreover, the tokenizer keeps a dictionary, `word_index`, from which we can get the number of elements in our vocabulary. In this case, we only have a few elements, but if we has passed a whole book, that function would be handy.

> NOTE: We should add `1` to the length of the `word_index` dictionary to get the correct number of elements.

In [ ]:
# get the number of elements in the vocabulary
vocab_size = len(tokenizer.word_index) + 1

We are now ready to process our features. First, we transform each character sequence (i.e., `sequence`, `structure`, `predicted_loop_type`) into number sequences and concatenate them together. Then, we add the previously extracted `bpps_*` features on top. The resulting shape should be `(num_examples, 107, 6)`.

In [ ]:
def process_features(df):
    df = df.copy()
    
    # get the features
    sequence_sentences = np.array(df["sequence"].values.tolist())
    structure_sentences = np.array(df["structure"].values.tolist())
    loop_sentences = np.array(df["predicted_loop_type"].values.tolist())
    
    # transform character sequences into number sequences
    sequence_tokens = tokenizer.texts_to_sequences(sequence_sentences)
    structure_tokens = tokenizer.texts_to_sequences(structure_sentences)
    loop_tokens = tokenizer.texts_to_sequences(loop_sentences)
    
    # concatenate the tokenized sequences
    sequences = np.stack((sequence_tokens, structure_tokens, loop_tokens), axis=1)
    sequences =  np.transpose(sequences, (0, 2, 1))
    
    # add the `bpps` features on top
    bpps_sum = np.array(df['bpps_sum'].to_list())[:,:,np.newaxis]
    bpps_max = np.array(df['bpps_max'].to_list())[:,:,np.newaxis]
    bpps_nb = np.array(df['bpps_nb'].to_list())[:,:,np.newaxis]
    
    return np.concatenate([sequences, bpps_sum, bpps_max, bpps_nb], 2)

In the same way we process the labels. We should just extract them and transform them into the correct shape. The resulting shape should be `(num_examples, 68, 5)`.

In [ ]:
def process_labels(df):
    df = df.copy()
    
    labels = np.array(df[pred_cols].values.tolist())
    labels = np.transpose(labels, (0, 2, 1))
    
    return labels

Before running our dataset from the prerpocess function, we need to add the extra info to the dataframe. Also, we need to separate the public and private test dataframes.

In [ ]:
train_df['bpps_sum'] = read_bpps_sum(train_df)
train_df['bpps_max'] = read_bpps_max(train_df)
train_df['bpps_nb'] = read_bpps_nb(train_df)

test_df['bpps_sum'] = read_bpps_sum(test_df)
test_df['bpps_max'] = read_bpps_max(test_df)
test_df['bpps_nb'] = read_bpps_nb(test_df)

In [ ]:
public_test_df = test_df.query("seq_length == 107")
private_test_df = test_df.query("seq_length == 130")

We are now ready to process the data set and make the features ready to be consumed by the model.

In [ ]:
x_train = process_features(train_df)
y_train = process_labels(train_df)

# Define and train the model

In [ ]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
         tf.keras.layers.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def build_model(vocab_size, seq_length=TRAIN_SEQUENCE_LENGTH, pred_len=68,
                embed_dim=EMBED_DIM,
                hidden_dim=HIDDEN_DIM, dropout=DROPOUT, sp_dropout=SP_DROPOUT):
    inputs = tf.keras.layers.Input(shape=(seq_length, 6))
    
    # split categorical and numerical features and concatenate them later.
    cat_features = inputs[:, :, :3]
    num_features = inputs[:, :, 3:]

    embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(cat_features)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    
    concatenated = tf.keras.layers.concatenate([reshaped, num_features], axis=2)
    
    hidden = tf.keras.layers.SpatialDropout1D(sp_dropout)(concatenated)
    
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model

In [ ]:
model = build_model(vocab_size)

In [ ]:
model.summary()

In [ ]:
class MeanColumnwiseRMSE(tf.keras.losses.Loss):
    def __init__(self, name='MeanColumnwiseRMSE'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [ ]:
model.compile(tf.optimizers.Adam(learning_rate=LR), loss=MeanColumnwiseRMSE())

In [ ]:
history = model.fit(x_train, y_train, validation_split=.1,
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2,
          callbacks=[
              tf.keras.callbacks.ModelCheckpoint("model_v0.3.0.h5", save_best_only=True)
          ])

In [ ]:
model.save('saved_model_v0.3.0')

## Evaluate the model

In [ ]:
public_test_data = process_features(public_test_df)
private_test_data = process_features(private_test_df)

In [ ]:
model_public = build_model(vocab_size, seq_length=107, pred_len=107)
model_private = build_model(vocab_size, seq_length=130, pred_len=130)

model_public.load_weights("model_v0.3.0.h5")
model_private.load_weights("model_v0.3.0.h5")

In [ ]:
public_preds = model_public.predict(public_test_data)
private_preds = model_private.predict(private_test_data)

# Submission

In [ ]:
preds_ls = []

for df, preds in [(public_test_df, public_preds), (private_test_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

In [ ]:
submission = sample_submission_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False)